# [Research Task - Create visuals for PUC 99314.11 leg report](https://github.com/cal-itp/data-analyses/issues/1656)
1. line graph of each metric (UPT, VRM, PMT) by agency
- x-axis is year
- y-axis is metric
- each line is an agency
- dotted line is average metric for all agencies in the year

2. line graph of each metric, by district
- similar to above
- each line is a district
- dotted line is average metrics for all districts the year

3. line graph of each metric, by mode
- similar to above
- each line is a mode
- dotter line is average metric for all modes in the year

Maybe try a box plot to show min/max/average for each metric?

## NTD Policy Manual for collecting UPT and PMT

### NTD Full Reporting Policy Manual 
However, FTA recognizes that certain statistics are challenging to collect and can drastically increase the reporting burden for transit agencies. To assist reporters who would find conducting 100 percent count burdensome, `transit agencies may estimate Unlinked Passenger Trips (UPT) and PMT through sampling`. The NTD provides a sampling method and sampling guidance on the NTD website.

### NTD Full Reporting Policy Manual & NTD Reduced Reporting Polict Manual
Collecting Service Consumed Data Transit agencies must report actual data on the Annual Report for all service data except UPT and PMT. `Only Full Reporters report PMT data to the NTD.` For these two data points, agencies may provide an estimate but only if the actual 100 percent data are not reliably collected and routinely processed.



In [1]:
import altair as alt
import pandas as pd
from calitp_data_analysis.sql import get_engine, to_snakecase, query_sql
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas
@cache
def gcs_pandas():
    return GCSPandas()

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = '{:,.2f}'.format

## Data querying, comparing, cleaning

### warehouse query

In [2]:
# metric_list = [
#     "pmt",
#     "upt",
#     "vrh",
#     # "opexp_total" # not needed for this project
# ]

# # empty list for appending DFs
# df_list = []

# for metric in metric_list:
#         query = f"""
#         SELECT
#           ntd_id,
#           source_agency,
#           agency_status,
#           primary_uza_name,
#           uza_population,
#           uza_area_sq_miles,
#           year,
#           mode,
#           type_of_service,
#           reporter_type,
#           SUM({metric}) AS total_{metric},
#         FROM
#           `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
#         WHERE
#           source_state = "CA"
#           AND year BETWEEN 2018 AND 2023
#         GROUP BY
#           ntd_id,
#           source_agency,
#           agency_status,
#           primary_uza_name,
#           uza_population,
#           uza_area_sq_miles,
#           year,
#           mode,
#           type_of_service,
#           reporter_type
#         """
#         # create df
#         metric = query_sql(query, as_df=True)

#         # append df to list
#         df_list.append(metric)

# # unpack list into separate DFs
# ntd_pmt, ntd_upt, ntd_vrh = df_list

In [3]:
# get districts for ntd ID


# for metric in metric_list:
#         query = f"""
#         SELECT
#           `mart_transit_database.dim_organizations`.`key` AS `key`,
#           `mart_transit_database.dim_organizations`.`source_record_id` AS `source_record_id`,
#           `mart_transit_database.dim_organizations`.`name` AS `name`,
#           `mart_transit_database.dim_organizations`.`ntd_id_2022` AS `ntd_id_2022`,
#           `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_name` AS `county`,
#           `Dim_County_Geography___County_Geography_Key`.`caltrans_district` AS `caltrans_district`
#         FROM
#           `mart_transit_database.dim_organizations`

#         LEFT JOIN `mart_transit_database.bridge_organizations_x_headquarters_county_geography` AS `Bridge_Organizations_X_Headquarters_County_Geography___Key` ON `mart_transit_database.dim_organizations`.`key` = `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`organization_key`
#           LEFT JOIN `mart_transit_database.dim_county_geography` AS `Dim_County_Geography___County_Geography_Key` ON `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_key` = `Dim_County_Geography___County_Geography_Key`.`key`
#         WHERE
#           (
#             `mart_transit_database.dim_organizations`.`_is_current` = TRUE
#           )

#            AND (
#             `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NOT NULL
#           )
#           AND (
#             (
#               `mart_transit_database.dim_organizations`.`ntd_id_2022` <> ''
#             )

#             OR (
#               `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NULL
#             )
#           )
#           AND (
#             `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`_is_current` = TRUE
#           )
#           AND (
#             `Dim_County_Geography___County_Geography_Key`.`_is_current` = TRUE
#           )
#         """
#         # create df
#         ntd_id_x_district = query_sql(query, as_df=True)
        
# ntd_id_x_district["caltrans_district"] = ntd_id_x_district["caltrans_district"].astype("str")

In [4]:
# merge_on_col = [
#     "ntd_id",
#     "year",
#     "source_agency",
#     "agency_status",
#     "primary_uza_name",
#     "uza_population",
#     "uza_area_sq_miles",
#     "mode",
#     "type_of_service",
#     "reporter_type",
# ]

# merge_1 = ntd_vrh.merge(ntd_upt, on=merge_on_col, how="inner")
# # merge_2 = merge_1.merge(ntd_vrh, on=merge_on_col, how = "inner")

# ntd_metrics_merge = merge_1.merge(ntd_pmt, on=merge_on_col, how="inner")

### data from other report

In [5]:
# gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
# ntd_name = "ntd_operator_data_18_23.parquet"

# ntd_all_metrics = pd.read_parquet(f"{gcs_path}{ntd_name}")

### compare datasets

In [6]:
# display(
#     ntd_all_metrics.info(), ntd_metrics_merge.info()  # mode/service is aggregated up
# )

In [7]:
# display(
#     ntd_all_metrics["ntd_id"].nunique()
#     == ntd_metrics_merge["ntd_id"].nunique(),  # TRUE, same count of unique values
#     set(ntd_all_metrics["ntd_id"].unique())
#     == set(ntd_metrics_merge["ntd_id"].unique()),  # TRUE, same unique NTD_IDs
# )

In [8]:
# display(
#     ntd_all_metrics["ntd_id"].nunique(),
#     ntd_metrics_merge["ntd_id"].nunique()
# )

In [9]:
# metric_cols = ["total_upt", "total_vrh", "total_upt"]

# for metric in metric_cols:
#     print(
#         ntd_all_metrics[metric].sum() == ntd_metrics_merge[metric].sum()
#     )  # TRUE sum of each metrics are equal

### merge in the district numbers to ntd_metric_merge

In [10]:
# ntd_metrics_merge = ntd_metrics_merge.merge(
#     ntd_id_x_district[["ntd_id_2022","county","caltrans_district"]],
#     left_on = "ntd_id",
#     right_on = "ntd_id_2022",
#     how="inner",
#     indicator=True
# )

In [11]:
# ntd_metrics_merge[ntd_metrics_merge["caltrans_district"].isna()].head()

## save out data

In [12]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
# ntd_metrics_merge.to_parquet(f"{gcs_path}puc_analysis_data_2025_12_9.parquet")

### read in cleaned ata

In [13]:
ntd_metrics_merge = gcs_pandas().read_parquet(f"{gcs_path}puc_analysis_data.parquet") #puc_analysis_data.parquet is initial analysis data
ntd_metrics_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4152 entries, 0 to 4151
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ntd_id             4002 non-null   object  
 1   source_agency      4152 non-null   object  
 2   agency_status      4152 non-null   object  
 3   primary_uza_name   4152 non-null   object  
 4   uza_population     4152 non-null   int64   
 5   uza_area_sq_miles  4152 non-null   float64 
 6   year               4152 non-null   int64   
 7   mode               4152 non-null   object  
 8   service            4152 non-null   object  
 9   reporter_type      4152 non-null   object  
 10  total_vrh          2623 non-null   float64 
 11  total_upt          2623 non-null   float64 
 12  total_pmt          2623 non-null   float64 
 13  ntd_id_2022        3798 non-null   object  
 14  county             3798 non-null   object  
 15  caltrans_district  3798 non-null   object  
 16  _merge     

**everything matches, moving with `ntd_metrics_merge` since its has mode/service**

In [14]:
cort_merge_filname = "ntd_cohort_data_2026-01-26.parquet"
ntd_cohort_merge = gcs_pandas().read_parquet(f"{gcs_path}{cort_merge_filname}")

In [16]:
ntd_cohort_merge.head()

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt,ntd_id_2022,county,caltrans_district,urban_rural,cohort,metric,_merge
0,90003,San Francisco Bay Area Rapid Transit District (BART),Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MG,PT,Full Reporter,"19,815.00","886,515.00","2,819,118.00",90003,San Francisco,4,Urban,Group A,Farebox Recovery Ratio,both
1,90003,San Francisco Bay Area Rapid Transit District (BART),Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MG,PT,Full Reporter,"19,815.00","886,515.00","2,819,118.00",90003,San Francisco,4,Urban,Group B,Local Funding % Change vs 2019,both
2,90003,San Francisco Bay Area Rapid Transit District (BART),Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,Urban,Group A,Farebox Recovery Ratio,both
3,90003,San Francisco Bay Area Rapid Transit District (BART),Active,"San Francisco--Oakland, CA",3515933,513.80,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,Urban,Group B,Local Funding % Change vs 2019,both
4,90003,San Francisco Bay Area Rapid Transit District (BART),Active,"San Francisco--Oakland, CA",3515933,513.80,2021,MG,PT,Full Reporter,"17,819.00","112,981.00","359,280.00",90003,San Francisco,4,Urban,Group B,Farebox Recovery Ratio,both


In [15]:
### save cleaned data to csv
# ntd_metrics_merge.to_csv(f"{gcs_path}puc_analysis_data_2025_12_9.csv")

## Group aggregations

In [18]:
# melt big DF so all columns are under 1 column.
group_list_melt = [
    "source_agency",
    "year",
    "ntd_id",
    "reporter_type",
    "caltrans_district",
    "mode",
    "service"
]

value_cols = ["total_upt", "total_vrh", "total_pmt"]

melt = pd.melt(
    ntd_metrics_merge,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="metric",
    value_name="metric_value",
    ignore_index=True,
)

In [ ]:
### save melted data to csv
# melt.to_csv(f"{gcs_path}puc_analysis_data_melt_2025_12_9.csv")

In [19]:
# What does group/agg the melted DF look like?
group_list_agg = [
    "source_agency",
    "year",
    "ntd_id",
    "reporter_type",
    "caltrans_district",
]
vrh_total = (
    melt[melt["metric"] == "total_vrh"]
    .groupby(group_list_agg)["metric_value"]
    .sum()
    .reset_index()
).rename(columns={"metric_value": "total_vrh"})

upt_total = (
    melt[melt["metric"] == "total_upt"]
    .groupby(group_list_agg)["metric_value"]
    .sum()
    .reset_index()
).rename(columns={"metric_value": "total_upt"})

passenger_total = (
    melt[melt["metric"] == "total_pmt"]
    .groupby(group_list_agg)["metric_value"]
    .sum()
    .reset_index()
).rename(columns={"metric_value": "total_pmt"})

yearly_totals = (
    ntd_metrics_merge.groupby(["year"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
) 

agency_totals = (
    ntd_metrics_merge.groupby(["year","source_agency"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

district_totals = (
    ntd_metrics_merge.groupby(["caltrans_district","year"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

mode_totals = (
    ntd_metrics_merge.groupby(["mode","year"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

In [ ]:
# how many rows have zero PMT?
len(passenger_total[passenger_total["total_pmt"] == 0])

### chart functtion with mean line

In [20]:
y_col = "1"
x_col = "2"
color_col= "3red"
tooltip= [y_col,x_col]
if color_col:
    tooltip.append(color_col)
tooltip

['1', '2', '3red']

In [21]:
type(tooltip)

list

In [22]:
def make_chart(data, x_col, y_col, title, color_col = False):
    tooltip_list=[y_col, x_col]
    if color_col:
        tooltip_list.append(color_col)
    
    chart = (alt.Chart(data)
        .mark_line(point=True)
        .encode(
            x=alt.X(x_col, axis = alt.Axis(labelFontSize=15, titleFontSize=15)),
            y=alt.Y(f"{y_col}:Q", title=f"{y_col}", axis=alt.Axis(labelFontSize=15, titleFontSize=15)),
            tooltip= tooltip_list,
            # legend = alt.Legend(labelFontSize=15, titleFontSize=15),
            # color = color_col if color_col else alt.Undefined,
            color=alt.Color(color_col, legend = alt.Legend(labelFontSize=15, titleFontSize=15)) if color_col else alt.Undefined
        )
        .properties(
            title= title,
            height=800,
            width="container",
        )
        .interactive()
    )

    # line for average
    baseline= pd.DataFrame({
    "baseline":[data[y_col].mean()]
    })
    
    line = (
        alt.Chart(baseline)
        .mark_rule(color = "red", strokeWidth=5, strokeDash=[10, 5], point=True,)
        .encode(
            y=alt.Y(f"baseline:Q",axis=alt.Axis(format=",.0f", orient="left")),
            tooltip=[alt.Tooltip(f"baseline")],
             color=alt.Color("baseline", 
                             legend = alt.Legend(title="baseline", labelFontSize=15, titleFontSize=15)
                            )
            )
        
    )


    combo = alt.layer(chart, line).resolve_scale(y="shared")

    return display(combo)

## Overall Totals

### Metric grand total per year

In [23]:
for col in yearly_totals.columns[1:]:
    yearly_avg = format(yearly_totals[col].mean(),",.2f")
    
    print(f"\nAverage {col} per  by year: {yearly_avg}"),
    make_chart(
        data = yearly_totals, 
        y_col = col,
        x_col = "year:N",
        title = f"Grand Total {col} per year",
    )



Average total_upt per  by year: 931,903,599.83


alt.LayerChart(...)


Average total_vrh per  by year: 41,186,329.17


alt.LayerChart(...)


Average total_pmt per  by year: 5,381,393,522.33


alt.LayerChart(...)

#### Boxplot of each metric grand total per year

In [ ]:
all_totals_dict = {
    "total_vrh": vrh_total,
    "total_upt": upt_total,
    "total_pmt": passenger_total,
}

# Boxplot
# removing zero-values to see what happens
for col, df in all_totals_dict.items():
    box_plot = (
        alt.Chart(df[df[col] != 0])
        .mark_boxplot(extent="min-max")
        .encode(
            x="year:N",
            y=alt.Y(col, axis=alt.Axis(format=",.0f", labelFontSize=15, titleFontSize=15)),
            # row = "reporter_type",
            tooltip=["source_agency", alt.Tooltip(col, format=",.f"), "year"],
        )
        .interactive()
        .properties(title=col, height=800, width="container")
    )

    display(
        f"Number of Agencies that reported zero {col}: {df[df[col]==0].ntd_id.nunique()}",
        box_plot.resolve_scale(y="independent"),
    )

### Metrics grand total by district, per year

In [ ]:
for col in district_totals.columns[2:]:
    district_avg = format(district_totals[col].mean(),",.2f")
    
    print(f"\nAverage {col} per  by year: {district_avg}"),
    make_chart(
        data = district_totals.astype({"caltrans_district":"int"}),
        y_col = col,
        x_col = "year:N",
        color_col = "caltrans_district:N",
        title = f"{col} by district per year"
    )

#### Box Plot of metric per district

In [ ]:
# Boxplot
# removing zero-values to see what happens
for col in district_totals.columns[2:]:
    box_plot = (
        alt.Chart(district_totals[district_totals[col] != 0])
        .mark_boxplot(extent="min-max")
        .encode(
            x="caltrans_district:N",
            y=col,
            # row = "reporter_type",
            tooltip=[col, "year"],
        )
        .interactive()
        .properties(title=f"Box Plot of {col} per district", height=200, width=1000)
    )

    display(
        f"\nNumber of rows that reported zero {col}: {district_totals[district_totals[col]==0][col].count()}",
        box_plot.resolve_scale(y="independent"),
    )

### Metrics grand total by agency, per year

In [ ]:
agency_avg = format(agency_totals[col].mean(),",.2f")

for col in agency_totals.columns[2:]:
    agency_avg = format(agency_totals[col].mean(),",.2f")
    
    print(f"\nAverage {col} per agency by year: {agency_avg}"),
    make_chart(
        data = agency_totals,
        y_col = col,
        x_col = "year:N",
        color_col = "source_agency:N",
        title = f"{col} per agency by year"
    )

### Metrics grand total by mode, per year

In [ ]:
for col in mode_totals.columns[2:]:
    mode_avg = format(mode_totals[col].mean(),",.2f")
    
    print(f"\nAverage {col} per mode by year: {mode_avg}"),
    make_chart(
        data = mode_totals,
        y_col = col,
        x_col = 'year:N',
        color_col = "mode:N",
        title = f"{col} per Mode by year",
    )
    

## Additional Comments

> “The pandemic had the most severe effects in more `urbanized Caltrans districts` (e.g., District 4: Bay Area and District 7: Los Angeles and Ventura Counties), where `unlinked passenger trips` and `passenger miles traveled` fell dramatically due to reduced commuting and widespread office closures. In `smaller districts`, ridership remained steadier, reflecting a customer base more reliant on transit for essential travel rather than commuting. 
>
>Recovery since 2021 has been uneven across the state. Although all districts have seen ridership and passenger miles rise from their pandemic lows, none have returned to `FY 2018–2019` highs. Caltrans District 7 (Los Angeles) and District 4 (Bay Area) have experienced the steepest declines and slowest recovery. Overall, `urbanized districts` drive the statewide totals, with their ridership swings dominating the overall trend. `Rural and small-agency districts`, however, exhibit much less volatility, underscoring the role of transit in those regions as an essential service rather than one tied primarily to commuting downtown cores.”


### District 4 and District 7 UPT / PMT 

In [17]:
for col in ["total_upt","total_pmt"]:
    make_chart(
        data = district_totals[
            district_totals["caltrans_district"].isin(["4","7"])
            ].astype({"caltrans_district":"int"}),
        y_col = col,
        x_col = "year:N",
        color_col = "caltrans_district:N",
        title = f"{col} for 'Urbanized District' (4 and 7) per year"
    )

NameError: name 'make_chart' is not defined

### Remaining District UPT / PMT

In [ ]:
for col in ["total_upt","total_pmt"]:
    make_chart(
        data = district_totals[~district_totals["caltrans_district"].isin(["4","7"])].astype({"caltrans_district":"int"}),
        y_col = col,
        x_col = "year:N",
        color_col = "caltrans_district:N",
        title = f"{col} for 'Rural/Smaller' Districts (1 to 12, exluding 4 and 7) per year"
    )

# Cohort Analysis

In [36]:
cohort_merge_farebox = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Farebox Recovery Ratio"]
cohort_merge_funding = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Local Funding % Change vs 2019"]

group_list_melt = [
    "source_agency",
    "year",
    "ntd_id",
    "caltrans_district",
    "mode",
    "type_of_service",
    "urban_rural",
    "cohort",
    "metric"
]

value_cols = ["total_upt", "total_vrh", "total_pmt"]

cohorts = [
    "Group A",
    "Group B",
    "Group C"
]
melt_farebox = pd.melt(
    cohort_merge_farebox,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

melt_funding = pd.melt(
    cohort_merge_funding,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

## Farebox

In [64]:
melt_farebox[melt_farebox["urban_rural"]=="Urban"].groupby(["cohort","urban_rural","ntd_metric"])["ntd_metric_value"].agg(["mean","median"])

mean       median
cohort  urban_rural ntd_metric                           
Group A Urban       total_pmt  38,244,270.43 5,094,241.00
                    total_upt   5,253,039.75   433,544.00
                    total_vrh     226,236.17    61,650.50
Group B Urban       total_pmt  22,940,119.87 2,443,033.00
                    total_upt   4,209,452.44   178,277.00
                    total_vrh     173,985.25    34,337.00
Group C Urban       total_pmt   8,609,814.47   572,356.00
                    total_upt   1,291,571.46    51,402.00
                    total_vrh      64,371.25    14,579.00

In [54]:
melt_farebox_agg = melt_farebox[
                (melt_farebox["urban_rural"] =="Urban")
                & (melt_farebox["ntd_metric"] == metric)
                # & (melt_farebox["cohort"] == cohort)
                ].groupby(["urban_rural","cohort","year"]).agg({"ntd_metric_value":"mean"}).reset_index()

### Urban 

In [55]:
for metric in value_cols:
    print(f"\nAverage {metric} per year")
    yearly_avg = format(melt_farebox[melt_farebox["ntd_metric"]==metric]["ntd_metric_value"].mean(),",.2f")
    make_chart(
        data = melt_farebox_agg,
        y_col = "ntd_metric_value",
        x_col = "year:N",
        title = f"Urban {metric} Average",
        color_col = "cohort"
        
    )


Average total_upt per year


alt.LayerChart(...)


Average total_vrh per year


alt.LayerChart(...)


Average total_pmt per year


alt.LayerChart(...)

line chart
data == total UPT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


line chart
data == total PMT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


line chart
data == total VRM
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


### Rural 

line chart
data == total UPT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


line chart
data == total PMT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group

line chart
data == total VRM
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group

## Funding Expended

In [66]:
melt_funding[melt_funding["urban_rural"]=="Rural"].groupby(["cohort","urban_rural","ntd_metric"])["ntd_metric_value"].agg(["mean","median"])

mean    median
cohort  urban_rural ntd_metric                    
Group A Rural       total_pmt        NaN       NaN
                    total_upt  74,330.06 18,504.00
                    total_vrh   9,132.80  5,580.00
Group B Rural       total_pmt  81,299.00 81,299.00
                    total_upt  43,752.21 18,720.50
                    total_vrh   8,838.74  4,809.00
Group C Rural       total_pmt        NaN       NaN
                    total_upt  40,246.54 17,623.50
                    total_vrh   8,731.70  6,106.00

### Urban 

line chart
data == total UPT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


line chart
data == total PMT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


line chart
data == total VRM
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


### Rural 

line chart
data == total UPT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group


line chart
data == total PMT
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group

line chart
data == total VRM
group == urban
x-axis == year
y axis == ntd metric value mean
each line is cohort group